In [1]:
import pandas as pd
import numpy as np
from script import *
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

import time

In [49]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [3]:
start = 20130101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
print(df_d.columns)
print(df_d.shape)

select * from stock_day where date>=20130101
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(2304627, 11)


In [4]:
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt'] /= 1e4
df = df_d

periods = [5,10,20,30,60,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]
print(df.columns)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma', 'close/5ma', 'close/10ma', 'close/20ma', 'close/30ma',
       'close/60ma', 'close/120ma', 'close/250ma'],
      dtype='object')


In [7]:
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
df_r = pd.read_parquet(r"database\return_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

In [15]:
print(df.index.is_lexsorted())
# df.groupby('code').apply(lambda x:np.arange(1,len(x+1))).head()
df['tmp'] = 1
df['idx'] = df.reset_index('code').groupby('code')['tmp'].expanding(1).sum().astype('int')
print(df['idx'].head())
del df['tmp']

In [16]:
index = df_d.index[(df_d['close/120ma']<=0.69)
                 & (df_d['close/30ma']<=0.778)
                 & (df_d['close/60ma']<=0.65)
                 & (df_d['close/60ma']>=0.55)
                ]
print(len(index))

1261


In [30]:
df_target = df.loc[index]
print(df_target.shape)
for i in range(1,6):
    df_target['idx_diff{}'.format(i)] = df_target.groupby('code')['idx'].diff(i)
    # 打印连续超跌i天的样本数量
    print('idx_diff{0}=={0}:'.format(i),(df_target['idx_diff{}'.format(i)]==i).sum())
    
df_target = df_target.join(df_r,how='inner',rsuffix='r_')
# 检查拼表后数据是否完整
print(df_target.shape)

(1261, 26)
idx_diff1==1: 564
idx_diff2==2: 305
idx_diff3==3: 179
idx_diff4==4: 117
idx_diff5==5: 80
(1261, 38)


In [50]:
# 统计连续超跌i天后次日买入的总收益率与各年收益率情况
for i in range(1,6):
    print('idx_diff{0}=={0}'.format(i))
    print(df_target[df_target['idx_diff{}'.format(i)]==i].reset_index('code')['r'].resample('Y').agg(agg_operations))
    print('-'*5)
    print(df_target[df_target['idx_diff{}'.format(i)]==i]['r'].agg(agg_operations))
    print('-'*10+'\n')

# 统计只超跌一天的收益率情况（注意这里实际上用到了未来数据，但依然有借鉴作用）
i=1
print('idx_diff1>{}'.format(i))
print(df_target[df_target['idx_diff{}'.format(i)]>i].reset_index('code')['r'].resample('Y').agg(agg_operations))
print('-'*5)
print(df_target[df_target['idx_diff{}'.format(i)]>i]['r'].agg(agg_operations))
print('-'*10+'\n')
print(len(df_target),(df_target['idx_diff{}'.format(i)]>i).sum(),(df_target['idx_diff{}'.format(i)]==i).sum())

# 结果显示，无论是整体收益率还是19年收益率，连续超跌天数越大，则表现越差。

idx_diff1==1
                mean    median       max       min       std  size
date                                                              
2015-12-31  0.082075  0.081982  0.655405 -0.333562  0.127632   277
2016-12-31  0.058145  0.047458  0.305152 -0.066667  0.065561    58
2017-12-31 -0.007091  0.000000  0.025126 -0.038410  0.029386     5
2018-12-31  0.010027  0.000000  0.224490 -0.189268  0.061744   175
2019-12-31 -0.016409 -0.003143  0.242424 -0.554711  0.142387    39
2020-12-31 -0.046639 -0.061890  0.002283 -0.065060  0.032650    10
-----
mean        0.048365
median      0.038402
max         0.655405
min        -0.554711
std         0.111786
size      564.000000
Name: r, dtype: float64
----------

idx_diff2==2
                mean    median       max       min       std  size
date                                                              
2015-12-31  0.058576  0.056522  0.551485 -0.221648  0.103748   107
2016-12-31  0.066191  0.055258  0.203170 -0.049231  0.061458    28
20

In [67]:
i=3
df_target[df_target['idx_diff{}'.format(i)]==i][['open','close','amt','buy_at','r','idx_diff5','idx_diff4','idx_diff3']]

open      close         amt     buy_at         r  \
code      date                                                                
000155.SZ 2017-12-28    9.59616    9.42368   37.683585    9.36096  0.025126   
          2017-12-29    9.36096    9.45504   21.666583    9.45504  0.014925   
          2018-01-02    9.45504    9.61184   24.836336    9.65888 -0.006494   
000156.SZ 2016-01-29   17.74239   18.60480   17.268203   18.63387  0.011440   
          2016-02-01   18.63387   18.21720   14.108151   18.26565  0.053050   
000503.SZ 2018-10-17  105.77235  104.63010   24.889928  106.91460  0.022080   
          2018-10-18  106.91460  103.25940   23.508771   98.99500  0.150000   
000558.SZ 2016-02-01   56.75670   57.15360   35.984142   57.31236  0.087258   
000750.SZ 2015-08-27   65.33033   68.23713   71.464925   69.03650  0.000000   
          2015-08-28   69.03650   71.94330   84.435181   71.57995 -0.049746   
          2015-08-31   71.57995   70.48990   66.097533   69.03650  0.031579   
000776.SZ 2015-08-27   60.77295   63.76500  154.290296   64.50075  0.000760   
000976.SZ 2018-06-29   18.84249   19.13633    8.610923   19.06287 -0.028902   
          2018-07-09   17.96097   18.32827    5.893124   18.43846  0.025896   
          2018-07-10   18.43846   18.40173    3.116681   18.21808  0.108871   
002073.SZ 2015-09-07  103.07595  106.20225   25.052122  103.53570  0.090586   
          2015-09-08  103.53570  112.91460   20.380061  115.12140 -0.019169   
002168.SZ 2015-09-07  120.99800  121.89650   19.426776  121.29750  0.139506   
002285.SZ 2016-02-01  102.12048   99.18249   14.921996   99.79035  0.065990   
002307.SZ 2015-07-16   25.08210   25.36920   26.480496   25.44750  0.048205   
          2015-07-17   25.44750   27.17010   28.132680   27.35280  0.019084   
002371.SZ 2015-09-08   90.71244   94.64000    7.795617   96.01228 -0.117792   
          2015-09-17   88.01520   85.41260    7.039350   85.88580  0.073278   
002411.SZ 2020-03-31   34.45282   32.16152   51.232793   32.32816       NaN   
          2020-04-01   32.32816   32.36982   38.996958        NaN       NaN   
002434.SZ 2018-10-22   38.27240   39.44600    9.188430   39.44600 -0.049587   
          2018-10-23   39.44600   38.33760    5.933960   38.40280  0.000000   
          2018-10-24   38.40280   38.46800    4.111978   37.49000  0.080000   
          2018-10-25   37.49000   37.81600    4.406924   38.40280  0.054329   
          2018-10-26   38.40280   37.62040    2.918031   37.88120  0.068847   
          2018-10-29   37.88120   37.03360    2.669241   37.29440  0.085664   
          2018-10-30   37.29440   37.81600    3.678458   36.96840  0.095238   
002463.SZ 2015-09-07   12.11210   12.16534   12.199751   12.00562 -0.028825   
002470.SZ 2019-05-10   19.03974   19.25319   34.012660   18.86898 -0.011312   
          2019-05-13   18.86898   18.61284   21.470520   18.27132 -0.016355   
          2019-05-14   18.27132   18.61284   18.988017   18.65553 -0.036613   
          2019-05-15   18.65553   19.21050   25.548514   19.08243 -0.058166   
          2019-05-16   19.08243   19.16781   17.851994   19.25319 -0.088692   
          2019-05-17   19.25319   18.35670   18.346684   17.97249  0.019002   
          2019-05-20   17.97249   17.54559   23.213202   17.54559  0.012165   
          2019-05-21   17.54559   18.35670   19.660413   18.31401 -0.030303   
          2019-05-22   18.31401   17.80173   14.568684   17.71635 -0.016867   
          2019-05-23   17.71635   17.63097   14.265261   17.75904 -0.064904   
          2019-05-24   17.75904   17.41752   10.435385   17.41752 -0.046569   
          2019-05-27   17.41752   17.92980   14.787409   17.84442 -0.069378   
          2019-05-28   17.84442   17.71635   11.056820   17.54559 -0.053528   
          2019-05-29   17.54559   17.71635    8.120078   17.54559 -0.053528   
002501.SZ 2018-08-07   28.10928   28.78752   14.006718   28.63680  0.050000   
          2018-08-08   28.63680   28.78752   16.714556   28.63680  0.047368  

In [66]:
i=1
result = pd.DataFrame(columns=agg_operations)
for k in [5,10,15,20,30,40]:
    result.loc['{}+null'.format(k),:] = df_target[(df_target['idx_diff{}'.format(i)]>k) 
                                                  | (df_target['idx_diff{}'.format(i)].isnull())]['r'].agg(agg_operations)
    result.loc['{}'.format(k),:] = df_target[df_target['idx_diff{}'.format(i)]>k]['r'].agg(agg_operations)
result.loc['null',:] = df_target[df_target['idx_diff{}'.format(i)].isnull()]['r'].agg(agg_operations)
# df_target[df_target['idx_diff{}'.format(i)]>20].reset_index('code')['r'].resample('A').agg(agg_operations)
result

,mean,median,max,min,std,size
5+null,0.0724989,0.0668596,1.20499,-0.399556,0.147737,515
5,0.0546385,0.0547521,1.20499,-0.270413,0.121309,141
10+null,0.076199,0.0710026,1.20499,-0.399556,0.152874,465
10,0.0632505,0.0553863,1.20499,-0.270413,0.137848,91
15+null,0.0775368,0.0730137,1.20499,-0.399556,0.154202,455
15,0.0689228,0.0617905,1.20499,-0.270413,0.14437,81
20+null,0.0782373,0.0735632,1.20499,-0.399556,0.154916,450
20,0.072374,0.0626435,1.20499,-0.270413,0.148263,76
30+null,0.0784337,0.0735286,1.20499,-0.399556,0.156627,439
30,0.0726551,0.0620564,1.20499,-0.270413,0.1587,65


In [73]:
mask = df_target['idx_diff1']==1
# df_target['tmp']=np.nan
df_target.loc[mask,'tmp']= df_target.loc[mask,'buy_at']
df_target.sort_index(inplace=True)
df_target['f2_open'] = df_target.groupby('code')['tmp'].shift(-1)

In [83]:
print(df_target['f2_open'].notnull().sum())
print(mask.sum())

df_target['r2'] = df_target['r']
df_target.loc[df_target.f2_open.notnull(),'r2'] = df_target[df_target.f2_open.notnull()]['f2_open']/df_target[df_target.f2_open.notnull()]['buy_at']-1

558
564


In [78]:
df_target[(~mask) & df_target['f2_open'].notnull()][df_r.columns]

,,buy_at,max,max_idx,idx,is_selled,sell_at,r
code,date,,,,,,,
000006.SZ,2016-01-27,221.77386,243.21520,1467.0,745,True,237.13482,0.069264
000011.SZ,2015-08-26,30.36912,34.29096,1346.0,643,True,35.39712,0.165563
000029.SZ,2016-01-26,17.47704,18.40713,1469.0,744,True,18.83973,0.077970
000043.SZ,2015-09-02,64.30350,66.90720,1361.0,648,True,66.43380,0.033129
000088.SZ,2015-08-26,48.31164,52.66404,1297.0,643,True,52.44642,0.085586
...,...,...,...,...,...,...,...,...
603600.SH,2016-01-29,43.45999,45.51614,251.0,250,True,43.81104,0.008078
603686.SH,2018-10-16,23.87639,24.06116,845.0,907,True,22.31611,-0.065348
603799.SH,2018-10-17,48.46140,53.63345,883.0,905,True,51.43710,0.061404


In [84]:
i=1
result2 = pd.DataFrame(columns=agg_operations)
for k in [5,10,15,20,30,40]:
    result2.loc['{}+null'.format(k),:] = df_target[(df_target['idx_diff{}'.format(i)]>k) 
                                                  | (df_target['idx_diff{}'.format(i)].isnull())]['r2'].agg(agg_operations)
    result2.loc['{}'.format(k),:] = df_target[df_target['idx_diff{}'.format(i)]>k]['r2'].agg(agg_operations)
result2.loc['null',:] = df_target[df_target['idx_diff{}'.format(i)].isnull()]['r2'].agg(agg_operations)
# df_target[df_target['idx_diff{}'.format(i)]>20].reset_index('code')['r'].resample('A').agg(agg_operations)
result2

,mean,median,max,min,std,size
5+null,0.0590113,0.0483871,1.20499,-0.399556,0.140872,515
5,0.0432531,0.0318584,1.20499,-0.146746,0.120468,141
10+null,0.060738,0.0485047,1.20499,-0.399556,0.146264,465
10,0.0432303,0.0291336,1.20499,-0.100088,0.139529,91
15+null,0.061671,0.0491287,1.20499,-0.399556,0.147625,455
15,0.0461268,0.0291336,1.20499,-0.100088,0.146849,81
20+null,0.0624714,0.0508242,1.20499,-0.399556,0.148238,450
20,0.0496901,0.0303218,1.20499,-0.100088,0.15074,76
30+null,0.0630349,0.0516532,1.20499,-0.399556,0.149539,439
30,0.0511912,0.0310901,1.20499,-0.100088,0.159676,65


In [85]:
df_target[df_target['r2']!=df_target['r']][['open','close','buy_at','f2_open','r','r2']]

,,open,close,buy_at,f2_open,r,r2
code,date,,,,,,
000006.SZ,2016-01-27,238.09488,223.05394,221.77386,221.45384,0.069264,-0.001443
000011.SZ,2015-08-26,33.38592,30.26856,30.36912,31.54232,0.165563,0.038631
000029.SZ,2016-01-26,19.03440,17.41215,17.47704,16.37391,0.077970,-0.063119
000043.SZ,2015-09-02,63.51450,62.80440,64.30350,59.96400,0.033129,-0.067485
000088.SZ,2015-08-26,52.95420,47.58624,48.31164,49.18212,0.085586,0.018018
...,...,...,...,...,...,...,...
603686.SH,2018-10-17,23.87639,23.67109,23.40420,22.31611,0.019298,-0.046491
603799.SH,2018-10-17,56.11320,49.48164,48.46140,47.62537,0.061404,-0.017251
603997.SH,2015-08-25,16.14432,16.14432,16.54592,16.26480,0.052791,-0.016990


In [88]:
df_target.loc[idx['603686.SH',:]][['open','close','f2_open','r2']+list(df_r.columns)]

,open,close,f2_open,r2,buy_at,max,max_idx,idx,is_selled,sell_at,r
date,,,,,,,,,,,
2018-10-12,25.35455,24.80024,NaN,-0.037252,24.80024,25.49826,843.0,905,True,23.87639,-0.037252
2018-10-16,24.69759,23.42473,23.40420,-0.019776,23.87639,24.06116,845.0,907,True,22.31611,-0.065348
2018-10-17,23.87639,23.67109,22.31611,-0.046491,23.40420,23.73268,846.0,908,True,23.85586,0.019298
2018-10-18,23.40420,22.87042,NaN,0.095676,22.31611,25.23137,848.0,909,True,24.45123,0.095676
